# Tag 4 - Guardian Data

In [3]:
import cred 
GUARDIAN_KEY = cred.GUARDIAN_KEY

In [20]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [6]:
# API Endpoint
API_ENDPOINT = 'http://content.guardianapis.com/search' 

In [12]:
PARAMS = {
    'api-key': GUARDIAN_KEY,
    'from-date': '2023-06-20', 
    'to-date': '2023-06-30', 
    'lang': 'en', 
    'production-office': 'uk',  
    'show-fields': 'wordcount,body,byline' ,
    'page-size': 50, # 50 Artikel je Seite
} 

response = requests.get(API_ENDPOINT, params=PARAMS) 
response_dict = response.json()['response']

In [14]:
response_dict['pages']

36

In [17]:
all_results = [] 
cur_page = 1 
total_pages = 1

while (cur_page <= total_pages) and (cur_page < 40): # with a fail safe 
    
    # Make a API request 
    PARAMS['page'] = cur_page 
    response = requests.get(API_ENDPOINT, params=PARAMS) 
    response_dict = response.json()['response'] 
    
    print(f"page: {cur_page} of {total_pages} - Articles (total): {response_dict['total']}")
    
    # Update our master results list 
    all_results += (response_dict['results']) 
    
    # Update our loop variables 
    total_pages = response_dict['pages'] 
    cur_page += 1 
    
    # sleep for 1 second
    time.sleep(1)
    

page: 1 of 1 - Articles (total): 1754
page: 2 of 36 - Articles (total): 1754
page: 3 of 36 - Articles (total): 1754
page: 4 of 36 - Articles (total): 1754
page: 5 of 36 - Articles (total): 1754
page: 6 of 36 - Articles (total): 1754
page: 7 of 36 - Articles (total): 1754
page: 8 of 36 - Articles (total): 1754
page: 9 of 36 - Articles (total): 1754
page: 10 of 36 - Articles (total): 1754
page: 11 of 36 - Articles (total): 1754
page: 12 of 36 - Articles (total): 1754
page: 13 of 36 - Articles (total): 1754
page: 14 of 36 - Articles (total): 1754
page: 15 of 36 - Articles (total): 1754
page: 16 of 36 - Articles (total): 1754
page: 17 of 36 - Articles (total): 1754
page: 18 of 36 - Articles (total): 1754
page: 19 of 36 - Articles (total): 1754
page: 20 of 36 - Articles (total): 1754
page: 21 of 36 - Articles (total): 1754
page: 22 of 36 - Articles (total): 1754
page: 23 of 36 - Articles (total): 1754
page: 24 of 36 - Articles (total): 1754
page: 25 of 36 - Articles (total): 1754
page: 26 o

In [18]:
all_results_df = pd.json_normalize(all_results)
all_results_df.head()

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName,fields.byline,fields.body,fields.wordcount
0,crosswords/prize/29111,crossword,crosswords,Crosswords,2023-06-30T23:00:03Z,"Prize crossword No 29,111",https://www.theguardian.com/crosswords/prize/2...,https://content.guardianapis.com/crosswords/pr...,False,pillar/lifestyle,Lifestyle,Paul,,0
1,crosswords/weekend/651,crossword,crosswords,Crosswords,2023-06-30T23:00:03Z,Weekend crossword No 651,https://www.theguardian.com/crosswords/weekend...,https://content.guardianapis.com/crosswords/we...,False,pillar/lifestyle,Lifestyle,Sy,,0
2,media/2023/jun/30/jeremy-clarkson-sun-article-...,article,media,Media,2023-06-30T21:00:00Z,Jeremy Clarkson’s Sun article on Meghan was se...,https://www.theguardian.com/media/2023/jun/30/...,https://content.guardianapis.com/media/2023/ju...,False,pillar/news,News,Jim Waterson and Alexandra Topping,<p>Jeremy Clarkson discriminated against the D...,885
3,society/2023/jun/30/nhs-plan-the-numbers-are-i...,article,society,Society,2023-06-30T20:53:57Z,"NHS plan: the numbers are impressive, but wher...",https://www.theguardian.com/society/2023/jun/3...,https://content.guardianapis.com/society/2023/...,False,pillar/news,News,Denis Campbell Health policy editor,"<p>It was on 8 November 2017 that Jeremy Hunt,...",546
4,sport/2023/jun/30/keely-hodgkinson-suffers-fir...,article,sport,Sport,2023-06-30T20:46:50Z,Keely Hodgkinson suffers first defeat of the y...,https://www.theguardian.com/sport/2023/jun/30/...,https://content.guardianapis.com/sport/2023/ju...,False,pillar/sport,Sport,Sean Ingle,"<p>During her brilliant young career, Keely Ho...",352


In [21]:
all_results_df['text'] = [BeautifulSoup(i, "html.parser").text for i in all_results_df['fields.body']]

In [24]:
# rename columns
all_results_df = all_results_df.rename(columns={'webTitle':'article_title',
                                                'webUrl':'article_url',
                                                'fields.byline':'article_author',
                                               'webPublicationDate': 'article_date'})

In [34]:
all_results_df_f = all_results_df[['article_date', 'sectionName', 'article_title', 'article_url', 'article_author', 'text']].copy()

In [35]:
all_results_df_f = all_results_df_f[all_results_df_f.text != ''].reset_index(drop=True)

In [36]:
all_results_df_f.to_csv('../data/guardian_articles_23_06_20-23_06_30.csv', index=False)